In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [2]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [3]:
# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

In [4]:
# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


In [5]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

In [6]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[ 0.1954,  0.1499, -0.2221],
        [ 0.0324,  0.4372, -0.2863]], requires_grad=True)
b:  Parameter containing:
tensor([-0.4896,  0.5448], requires_grad=True)


In [7]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [8]:
# Forward pass.
pred = linear(x)

In [9]:
# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

loss:  0.766990065574646


In [10]:
# Backward pass.
loss.backward()

In [11]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[-0.1221, -0.0342, -0.3245],
        [ 0.4167,  0.1309,  0.0797]])
dL/db:  tensor([0.0244, 0.5076])


In [12]:
# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

In [13]:
# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  0.7612648606300354


In [14]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

In [15]:
# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

In [16]:
# Convert the torch tensor to a numpy array.
z = y.numpy()

In [17]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

Files already downloaded and verified


In [18]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


In [19]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [20]:
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

In [22]:
# Mini-batch images and labels.
images, labels = next(data_iter)

In [23]:
# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

In [26]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # Initialize your dataset (e.g., a list of file paths or data)
        self.data = [...]  # Add the actual data or file paths here

    def __getitem__(self, index):
        # Example: Replace this with the actual logic to load and preprocess data
        # e.g., using numpy, PIL.Image, or other data processing tools
        image = np.load(self.data[index])  # Replace with actual data loading
        label = ...  # Replace with the corresponding label logic
        image = torch.tensor(image, dtype=torch.float32)  # Convert to tensor
        return image, label

    def __len__(self):
        # This should return the total number of samples in the dataset
        return len(self.data)

In [27]:
# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)


In [28]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

/Users/tharacaba/opt/anaconda3/envs/astro/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/tharacaba/opt/anaconda3/envs/astro/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

In [30]:
# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

In [31]:
# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

torch.Size([64, 100])


In [32]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

In [33]:
# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>